In [1]:
# Etapa 1: Query SQL para custo quadro

# Caminho físico do lakehouse de destino
path_destino = "abfss://ws_obras@onelake.dfs.fabric.microsoft.com/lk_obras.Lakehouse/Tables/tab_gold_fato_custo_quadro"

crateiolc = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoContabil.Lakehouse/Tables/crateiolc"
)
crateiolc.createOrReplaceTempView("crateiolc")

cpartida = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoContabil.Lakehouse/Tables/cpartida"
)
cpartida.createOrReplaceTempView("cpartida")

clancamento = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoContabil.Lakehouse/Tables/clancamento"
)
clancamento.createOrReplaceTempView("clancamento")

cconta = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoContabil.Lakehouse/Tables/cconta"
)
cconta.createOrReplaceTempView("cconta")

clcompl = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoContabil.Lakehouse/Tables/clcompl"
)
clcompl.createOrReplaceTempView("clcompl")

gccusto = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_InteligenciaNegocios.Lakehouse/Tables/gccusto"
)
gccusto.createOrReplaceTempView("gccusto")


df_resultado = spark.sql("""
WITH base AS (
    SELECT
        -- cpartida
        cp.CODCOLIGADA,
        cp.CODFILIAL,
        cp.IDLANCAMENTO,
        cp.IDPARTIDA,
        cp.LCTREF,
        cp.DOCUMENTO,
        cp.DATA,
        cp.DATAINCLU,
        cp.DATAINTEGRACAO,
        cp.RECMODIFIEDON,
        cp.USUARIO,
        cp.USUARIOALTER,
        cp.TIPOGERACAO,
        cp.INTEGRAAPLICACAO,
        cp.CODLOTE,
        cp.COMPLEMENTO,
        cp.CODHISTP,

        -- clancamento
        cl.DESCRICAO                                  AS CL_DESCRICAO,
        cl.CODLOTE                                    AS CL_LOTE,

        -- crateiolc
        cr.CODCONTA                                   AS CR_CODCONTA,
        cr.CODGERENCIAL                               AS CR_CODGERENCIAL,
        cr.VLRDEBITO                                  AS CR_VLRDEBITO,
        cr.VLRCREDITO                                 AS CR_VLRCREDITO,

        -- cconta e variações
        cta.DESCRICAO                                 AS NOMECONTA,
        cta.NATUREZA                                  AS NATUREZA_CONTA,
        cta.REDUZIDO                                  AS REDUZIDO,
        cta_dre.DESCRICAO                             AS DRE_DESC,
        cta_macro.DESCRICAO                           AS MACRO_DESC,

        -- gccusto / clcompl
        cc.CODREDUZIDO                                AS CODREDUZIDO_CC,
        cc.NOME                                       AS NOME_CC,
        co.INVESTIMENTO,

        -- derivados padronizados
        date_format(cp.DATA, 'yyyy/MM')               AS COMPETENCIA,
        year(cp.DATA)                                 AS ANO,
        month(cp.DATA)                                AS MES,
        substring(cr.CODCONTA, 1, 1)                  AS GRUPO,

        substring(cr.CODCONTA, length(cr.CODCONTA) - 14 + 1, 14) AS CODCONTA_14,

        trunc(cp.DATA, 'MM')                          AS DATA_COMP,

        substring(substring(cr.CODCONTA, length(cr.CODCONTA) - 14 + 1, 14), 1, 3)  AS COD_3,
        substring(substring(cr.CODCONTA, length(cr.CODCONTA) - 14 + 1, 14), 1, 9)  AS COD_9,
        substring(cr.CODGERENCIAL, 1, 4)                                         AS NEGOCIO,
        substring(cr.CODGERENCIAL, 1, 7)                                         AS DIVISAO

    FROM crateiolc cr
    JOIN cpartida  cp
      ON cr.CODCOLIGADA = cp.CODCOLIGADA
     AND cr.IDPARTIDA   = cp.IDPARTIDA
    JOIN clancamento cl
      ON cp.CODCOLIGADA = cl.CODCOLIGADA
     AND cp.IDLANCAMENTO = cl.IDLANCAMENTO
    JOIN cconta cta
      ON cta.CODCOLIGADA = 0
     AND cta.CODCONTA    = cr.CODCONTA
    LEFT JOIN cconta cta_dre
      ON cta_dre.CODCOLIGADA = 0
     AND cta_dre.CODCONTA = CASE
                                WHEN cr.CODCONTA LIKE '9%' THEN substring(cr.CODCONTA, 1, 5)
                                ELSE substring(cr.CODCONTA, 1, 3)
                            END
    LEFT JOIN cconta cta_macro
      ON cta_macro.CODCOLIGADA = 0
     AND cta_macro.CODCONTA = CASE
                                  WHEN cr.CODCONTA LIKE '9%' THEN substring(cr.CODCONTA, 1, 11)
                                  ELSE substring(cr.CODCONTA, 1, 9)
                              END
    LEFT JOIN clcompl co
      ON co.CODCOLIGADA = cp.CODCOLIGADA
     AND co.IDPARTIDA   = cp.IDPARTIDA
    LEFT JOIN gccusto cc
      ON cr.CODCOLIGADA  = cc.CODCOLIGADA
     AND cr.CODGERENCIAL = cc.CODCCUSTO
    WHERE
        cp.TIPOGERACAO <> 'A'
        AND cp.CODCOLIGADA IN (1, 2, 7)
        AND year(cp.DATA) BETWEEN year(current_date()) - 3 AND year(current_date())
        AND (
              substring(cr.CODCONTA, 1, 1) = '3'
              OR (substring(cr.CODCONTA, 1, 1) = '9' AND cl.DESCRICAO LIKE 'AG - Antecipação de Lucros%')
            )
        AND (
              ( substring(cr.CODGERENCIAL, 1, 4) = '1.03'
                OR cc.CODREDUZIDO IN ('2202D','1600A','1600I','01221','1221I','1160A','1161A','1162A','1162I','1.12.01.01221','1300A','1019I')
              )
           OR 
              ( substring(cr.CODGERENCIAL, 1, 4) IN ('1.04','1.05','1.06','1.07','1.08','1.09','1.10','1.11')
                OR cc.CODREDUZIDO IN ('1164A','1164I','1340A','1340I','1019I')
              )
            )
)
SELECT
    b.CODCOLIGADA,
    b.CODFILIAL,
    b.IDLANCAMENTO,
    b.IDPARTIDA,
    b.LCTREF,
    b.DOCUMENTO,
    b.DATA,
    b.DATAINCLU,
    b.DATAINTEGRACAO,
    b.RECMODIFIEDON,
    b.USUARIO,
    b.USUARIOALTER,

    b.COMPETENCIA,
    b.ANO,
    b.MES,
    b.GRUPO,
    b.CR_CODCONTA                        AS CODCONTA_ORIGINAL,
    b.CODCONTA_14                        AS CODCONTA,

    CASE
        WHEN b.CR_CODCONTA LIKE '3.1%' OR b.CR_CODCONTA LIKE '9.3.1%' THEN 'RECEITA TOTAL'
        ELSE 'CUSTO TOTAL'
    END AS CLASSIF,

    b.DATA_COMP,

    CASE
        WHEN b.CODCONTA_14 IN ('3.1.1.998.0001','3.1.1.001.0003','3.1.1.001.0004') THEN
            CASE -SUM(coalesce(b.CR_VLRDEBITO,0))
                WHEN 0 THEN '( + ) Provisoes Receita'
                ELSE '( – ) Reversões Receita'
            END
        ELSE
            CASE b.COD_3
                WHEN '3.1' THEN '( + ) Receita Faturada'
                WHEN '3.2' THEN '( – ) Impostos'
                ELSE '( – ) Custos'
            END
    END AS CLASSIFICACAO,

    concat(b.COD_3, ' - ',  b.DRE_DESC)  AS DRE,
    concat(b.COD_9, '   ',   b.MACRO_DESC) AS MACROCONTA,

    CASE
        WHEN b.COD_3 = '3.1' THEN '01. Receita'
        WHEN b.COD_3 = '3.2' THEN '02. Impostos'
        ELSE
            CASE b.COD_9
                WHEN '3.3.1.001' THEN '03.1 Pessoal + Encargos'
                WHEN '3.3.1.999' THEN '03.1 Pessoal + Encargos'
                WHEN '3.3.1.002' THEN '03.2 Indenizações'
                WHEN '3.3.1.003' THEN '04. Transporte de Pessoal'
                WHEN '3.3.1.004' THEN '05. Saúde'
                WHEN '3.3.1.005' THEN '06. Alimentação'
                WHEN '3.3.1.006' THEN '07. Equipamentos e Locações'
                WHEN '3.3.1.007' THEN '08. Veículos e Locações'
                WHEN '3.3.1.008' THEN '09. Escritório e Hospedagem'
                WHEN '3.3.1.009' THEN '10. Serviços de Terceiros'
                WHEN '3.3.1.010' THEN '11. Fretes e Carretos'
                WHEN '3.3.1.011' THEN '12. EPIs e Uniformes'
                WHEN '3.3.1.012' THEN '13. Ferramentas e Utensílios'
                WHEN '3.3.1.013' THEN '14. Materiais'
                WHEN '3.3.1.014' THEN '15. Utilidades e Serviços'
                WHEN '3.3.1.015' THEN '16. Gastos Gerais'
                WHEN '3.3.2.001' THEN '14. Materiais'
                WHEN '3.3.3.001' THEN '14. Materiais'
                WHEN '3.3.3.002' THEN '03.1 Pessoal + Encargos'
                WHEN '3.3.3.003' THEN '03.2 Indenizações'
                WHEN '3.3.3.004' THEN '04. Transporte de Pessoal'
                WHEN '3.3.3.005' THEN '05. Saúde'
                WHEN '3.3.3.006' THEN '06. Alimentação'
                WHEN '3.4.1.001' THEN '03.1 Pessoal + Encargos'
                WHEN '3.4.1.999' THEN '03.1 Pessoal + Encargos'
                WHEN '3.4.1.002' THEN '03.2 Indenizações'
                WHEN '3.4.1.003' THEN '04. Transporte de Pessoal'
                WHEN '3.4.1.004' THEN '05. Saúde'
                WHEN '3.4.1.005' THEN '06. Alimentação'
                WHEN '3.4.1.006' THEN '07. Equipamentos e Locações'
                WHEN '3.4.1.007' THEN '08. Veículos e Locações'
                WHEN '3.4.1.008' THEN '09. Escritório e Hospedagem'
                WHEN '3.4.1.009' THEN '10. Serviços de Terceiros'
                WHEN '3.4.1.010' THEN '11. Fretes e Carretos'
                WHEN '3.4.1.011' THEN '12. EPIs e Uniformes'
                WHEN '3.4.1.012' THEN '13. Ferramentas e Utensílios'
                WHEN '3.4.1.013' THEN '14. Materiais'
                WHEN '3.4.1.014' THEN '15. Utilidades e Serviços'
                WHEN '3.4.1.015' THEN '16. Gastos Gerais'
                WHEN '3.4.2.001' THEN '17. Outras Receitas Operacionais'
                ELSE 'XX. Verificar'
            END
    END AS GRUPO_MACRO,

    CASE
        WHEN b.COD_3 = '3.1' THEN 'RECEITA BRUTA'
        WHEN b.COD_3 = '3.2' THEN 'IMPOSTOS'
        ELSE
            CASE b.COD_9
                WHEN '3.3.1.001' THEN 'MÃO DE OBRA'
                WHEN '3.3.1.999' THEN 'MÃO DE OBRA'
                WHEN '3.3.1.002' THEN 'VERBA INDENIZATÓRIA'
                WHEN '3.3.1.013' THEN 'MATERIAIS'
                WHEN '3.4.1.001' THEN 'MÃO DE OBRA'
                WHEN '3.4.1.999' THEN 'MÃO DE OBRA'
                WHEN '3.4.1.002' THEN 'VERBA INDENIZATÓRIA'
                WHEN '3.4.1.013' THEN 'MATERIAIS'
                WHEN '3.3.2.001' THEN 'MATERIAIS'
                WHEN '3.3.3.001' THEN 'MATERIAIS'
                WHEN '3.3.3.002' THEN 'MÃO DE OBRA'
                WHEN '3.3.3.003' THEN 'VERBA INDENIZATÓRIA'
                ELSE 'OUTROS'
            END
    END AS GRUPOFX,

    b.REDUZIDO,
    b.TIPOGERACAO,
    b.INTEGRAAPLICACAO                        AS INTEGRACAO,
    b.CODLOTE,
    b.NOMECONTA,
    b.NATUREZA_CONTA                           AS NATUREZA,
    b.INVESTIMENTO,
    b.CL_DESCRICAO                             AS NOME_LANCAMENTO,
    b.CL_LOTE                                  AS LOTELC,
    b.COMPLEMENTO,
    b.CODHISTP                                  AS CODHIST,
    b.NEGOCIO,
    b.DIVISAO,
    b.CR_CODGERENCIAL                          AS CODCCUSTO,
    b.CODREDUZIDO_CC                           AS CODREDUZIDO,
    b.NOME_CC                                  AS NOME,

    -SUM(coalesce(b.CR_VLRDEBITO, 0))                                      AS VALORDEBITO,
     SUM(coalesce(b.CR_VLRCREDITO, 0))                                     AS VALORCREDITO,
     SUM(coalesce(b.CR_VLRCREDITO, 0) - coalesce(b.CR_VLRDEBITO, 0))       AS VALORSALDO,
     abs(SUM(coalesce(b.CR_VLRCREDITO, 0) - coalesce(b.CR_VLRDEBITO, 0)))  AS VALORABS,

    CASE
        WHEN (b.NEGOCIO IN ('1.04','1.05','1.06','1.07','1.08','1.09','1.10','1.11')
              OR b.CODREDUZIDO_CC IN ('1164A','1164I','1340A','1340I','1019I'))
        THEN 'OBRAS'
        ELSE 'CONTRATOS'
    END AS TIPO
FROM base b
GROUP BY
    b.CODCOLIGADA, b.CODFILIAL, b.IDLANCAMENTO, b.IDPARTIDA, b.LCTREF, b.DOCUMENTO,
    b.DATA, b.DATAINCLU, b.DATAINTEGRACAO, b.RECMODIFIEDON, b.USUARIO, b.USUARIOALTER,
    b.COMPETENCIA, b.ANO, b.MES, b.GRUPO, b.CR_CODCONTA, b.CODCONTA_14, b.DATA_COMP,
    b.COD_3, b.COD_9, b.DRE_DESC, b.MACRO_DESC,
    b.REDUZIDO, b.TIPOGERACAO, b.INTEGRAAPLICACAO, b.CODLOTE, b.NOMECONTA, b.NATUREZA_CONTA,
    b.INVESTIMENTO, b.CL_DESCRICAO, b.CL_LOTE, b.COMPLEMENTO, b.CODHISTP,
    b.NEGOCIO, b.DIVISAO, b.CR_CODGERENCIAL, b.CODREDUZIDO_CC, b.NOME_CC
""")

# Salva no Lakehouse de destino usando caminho físico
df_resultado.write.format("delta").mode("overwrite").save(path_destino)


StatementMeta(, 08c4cdac-3991-424d-a03c-048d1582791a, 3, Finished, Available, Finished)